In [3]:
# Install required packages
!pip install pycaret gradio

# Step 1: Import the necessary libraries
import pandas as pd
from pycaret.classification import *
import gradio as gr

# Step 2: Load the Wine Quality dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')


In [4]:
# Step 3: Set up the PyCaret classification experiment
exp = setup(data, target='quality', session_id=123)

# Step 4: Train the best model using compare_models()
best_model = compare_models()

,Description,Value
0,Session id,123
1,Target,quality
2,Target type,Multiclass
3,Target mapping,"3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5"
4,Original data shape,"(1599, 12)"
5,Transformed data shape,"(1599, 12)"
6,Transformed train set shape,"(1119, 12)"
7,Transformed test set shape,"(480, 12)"
8,Numeric features,11
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6702,0.5871,0.6702,0.6485,0.6494,0.4612,0.4667,0.4530
xgboost,Extreme Gradient Boosting,0.6622,0.5663,0.6622,0.6420,0.6479,0.4581,0.4607,0.3050
rf,Random Forest Classifier,0.6595,0.5761,0.6595,0.6323,0.6388,0.4444,0.4500,0.6230
lightgbm,Light Gradient Boosting Machine,0.6497,0.5725,0.6497,0.6277,0.6342,0.4365,0.4403,3.8540
gbc,Gradient Boosting Classifier,0.6345,0.0000,0.6345,0.6189,0.6231,0.4164,0.4186,1.8150
lr,Logistic Regression,0.6077,0.0000,0.6077,0.5782,0.5791,0.3493,0.3558,0.7340
lda,Linear Discriminant Analysis,0.5996,0.0000,0.5996,0.5814,0.5853,0.3572,0.3601,0.0660
ridge,Ridge Classifier,0.5880,0.0000,0.5880,0.4973,0.5318,0.2989,0.3103,0.1540
dt,Decision Tree Classifier,0.5872,0.4659,0.5872,0.5837,0.5821,0.3556,0.3572,0.1170
nb,Naive Bayes,0.5558,0.5033,0.5558,0.5628,0.5551,0.3141,0.3162,0.0710


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [5]:
# Step 5: Finalize the model (train on the full dataset)
final_model = finalize_model(best_model)

# Step 6: Save the model to disk
save_model(final_model, 'final_wine_model')

# Load the model back (for deployment, it's good practice)
model = load_model('final_wine_model')


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


In [6]:
# Step 7: Define the prediction function with debugging to inspect column names
def predict(fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol):
    try:
        # Prepare the input data as a DataFrame
        input_data = pd.DataFrame([[fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol]],
                                  columns=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'])

        # Debugging: Print input data
        print(f"Input Data:\n{input_data}")

        # Use the loaded model to make predictions
        prediction = predict_model(model, data=input_data)

        # Debugging: Print prediction output to inspect the columns
        print(f"Prediction Output:\n{prediction}")

        # Check if 'Label' column exists, else return first column
        if 'Label' in prediction.columns:
            return prediction['Label'][0]
        else:
            # Return the first column name dynamically
            return prediction.iloc[:, -1][0]  # Adjust this to return the last column (likely the prediction)

    except Exception as e:
        # If there's an error, return the error message for debugging
        return f"Error: {e}"



In [7]:
# Step 8: Create the Gradio Interface with proper input ranges
fixed_acidity = gr.Slider(minimum=4.6, maximum=15.9, value=7.4, label='Fixed Acidity')
volatile_acidity = gr.Slider(minimum=0.0, maximum=1.6, value=0.5, label='Volatile Acidity')
citric_acid = gr.Slider(minimum=0.0, maximum=1.0, value=0.0, label='Citric Acid')
residual_sugar = gr.Slider(minimum=0.9, maximum=15.5, value=5.0, label='Residual Sugar')
chlorides = gr.Slider(minimum=0.012, maximum=0.1, value=0.076, label='Chlorides')
free_sulfur_dioxide = gr.Slider(minimum=1.0, maximum=72.0, value=11.0, label='Free Sulfur Dioxide')
total_sulfur_dioxide = gr.Slider(minimum=6.0, maximum=289.0, value=34.0, label='Total Sulfur Dioxide')
density = gr.Slider(minimum=0.990, maximum=1.003, value=0.9968, label='Density')
pH = gr.Slider(minimum=2.74, maximum=4.01, value=3.18, label='pH')
sulphates = gr.Slider(minimum=0.33, maximum=2.0, value=0.56, label='Sulphates')
alcohol = gr.Slider(minimum=8.4, maximum=14.9, value=10.0, label='Alcohol')


In [8]:

# Create the Gradio interface with the trained model
gr.Interface(fn=predict,
             inputs=[fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol],
             outputs="text",
             title="Wine Quality Prediction").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4de80130d9e48c07cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
